In [4621]:
import nltk
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.experimental import enable_iterative_imputer
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.feature_selection import RFE, RFECV, f_regression, chi2
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import GridSearchCV, KFold, RandomizedSearchCV, HalvingGridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, recall_score, precision_score, f1_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.preprocessing import PowerTransformer, QuantileTransformer
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.neighbors import  KNeighborsClassifier
from datetime import datetime as dt
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder
from imblearn.combine import SMOTETomek
from collections import Counter
from mlxtend.feature_selection import ColumnSelector
import warnings
# Settings the warnings to be ignored
warnings.filterwarnings('ignore')

In [4622]:
# Displaying all columns
pd.set_option('display.max_columns', None)

#Display all data in columns
# pd.set_option("display.max_colwidth", None)

# np.set_printoptions(threshold=None, edgeitems=None, linewidth=None, suppress=None)
# pd.set_option("display.max_rows", None)
# pd.set_option("display.expand_frame_repr", True)
# pd.set_option('display.width', 1000)


In [4623]:
#Storing Accuracy

data_accuracy = pd.DataFrame(columns=['Algo', 'Accuracy(KFold)', 'Test', 'Train', 'Recall', 'Precision', 'F1', 'ROC_AUC_SCORE'])
def store(data):
    data_accuracy.loc[len(data_accuracy.index)] = data
    data_accuracy.to_csv('loan_prediction_metric_scores.csv')
    print(data_accuracy)



def get_info_pipeline(pipe_name):
    sub_pipe = pipe_name[:-1]
    transformed_df = pd.DataFrame(
        data=sub_pipe.fit_transform(X_train, y_train),
        columns=sub_pipe.get_feature_names_out(),
    )
    print(transformed_df)

In [4624]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [4625]:
df1 = pd.read_csv('tmdb_5000_credits.csv')
df2 = pd.read_csv('tmdb_5000_movies.csv')

In [4626]:
df1.head(1)

movie_id   title  \
0     19995  Avatar   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [4627]:
#Renaming the 'movie_id' col in df1 to 'id' of df2 

In [4628]:
df1.rename(columns = {'movie_id':'id'}, inplace = True)

In [4629]:
df1.head(1)

id   title  \
0  19995  Avatar   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [4630]:
#joining the credits amd movie dataframes

In [4631]:
df2 = pd.merge(df1, df2, on='id')

In [4632]:
df2.head(1)

id title_x  \
0  19995  Avatar   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [4633]:
#Drpping columns

In [4634]:
df2.columns

Index(['id', 'title_x', 'cast', 'crew', 'budget', 'genres', 'homepage', 'keywords', 'original_language', 'original_title', 'overview', 'popularity', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title_y', 'vote_average', 'vote_count'], dtype='object')

In [4635]:
df2.drop(columns=['status', 'runtime'], axis=1, inplace=True)

In [4636]:
df2.drop(columns=['title_x'], axis=1, inplace=True)

In [4637]:
df2.head(1)

id  \
0  19995   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [4638]:
df2.drop(columns=['title_y'], axis=1, inplace=True)

In [4639]:
df2.shape

(4803, 19)

In [4640]:
df2.head(1)

id  \
0  19995   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [4641]:
df2.drop(columns=['revenue', 'release_date'], axis=1, inplace=True)

In [4642]:
df2.drop(columns=['budget', 'homepage', 'popularity', 'spoken_languages', 'production_countries', 'vote_average', 'vote_count'], axis=1, inplace=True)

In [4643]:
df2.shape

(4803, 10)

In [4644]:
df2.head(1)

id  \
0  19995   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [4645]:
df2.isnull().sum()

id                        0
cast                      0
crew                      0
genres                    0
keywords                  0
original_language         0
original_title            0
overview                  3
production_companies      0
tagline                 844
dtype: int64

In [4646]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4803 entries, 0 to 4802
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id                    4803 non-null   int64 
 1   cast                  4803 non-null   object
 2   crew                  4803 non-null   object
 3   genres                4803 non-null   object
 4   keywords              4803 non-null   object
 5   original_language     4803 non-null   object
 6   original_title        4803 non-null   object
 7   overview              4800 non-null   object
 8   production_companies  4803 non-null   object
 9   tagline               3959 non-null   object
dtypes: int64(1), object(9)
memory usage: 412.8+ KB


In [4647]:
df2.drop(columns='production_companies', axis=1, inplace=True)

In [4648]:
df2.head(1)

id  \
0  19995   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [4649]:
df2.shape

(4803, 9)

In [4650]:
#Dropping null rows

In [4651]:
df2.isnull().sum()

id                     0
cast                   0
crew                   0
genres                 0
keywords               0
original_language      0
original_title         0
overview               3
tagline              844
dtype: int64

In [4652]:
df2.drop(columns='tagline', axis=1, inplace=True)

In [4653]:
df2.dropna(inplace=True)

In [4654]:
df2.head(1)

id  \
0  19995   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [4655]:
df2.isnull().sum()

id                   0
cast                 0
crew                 0
genres               0
keywords             0
original_language    0
original_title       0
overview             0
dtype: int64

In [4656]:
#Preprocessing

In [4657]:
##Genre

In [4658]:
df2.iloc[0, 3]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [4659]:
import ast

In [4660]:
def fetch_genres(string):
    ls = []
    for i in  ast.literal_eval(string):
        ls.append(i['name'])
    return ls

In [4661]:
df2['genres'] = df2['genres'].apply(fetch_genres)

In [4662]:
df2.head(1)

id  \
0  19995   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [4663]:
df2.iloc[3, 3]

['Action', 'Crime', 'Drama', 'Thriller']

In [4664]:
##Crew

In [4665]:
def fetch_dir_and_prod(string):
    ls = []
    for i in  ast.literal_eval(string):
        if i['job']=='Director':
            ls.append(i['name'])
            break
    return ls

In [4666]:
df2.head(1)

id  \
0  19995   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [4667]:
msg = " "
msg = df2.loc[4].crew

In [4668]:
msg

'[{"credit_id": "52fe479ac3a36847f813eaa3", "department": "Writing", "gender": 2, "id": 7, "job": "Screenplay", "name": "Andrew Stanton"}, {"credit_id": "52fe479ac3a36847f813ea65", "department": "Directing", "gender": 2, "id": 7, "job": "Director", "name": "Andrew Stanton"}, {"credit_id": "58c69e819251411d30017673", "department": "Crew", "gender": 2, "id": 7879, "job": "Thanks", "name": "John Lasseter"}, {"credit_id": "52fe479ac3a36847f813eac5", "department": "Production", "gender": 2, "id": 490, "job": "Producer", "name": "Colin Wilson"}, {"credit_id": "58c68f2b9251411d30016cf2", "department": "Production", "gender": 2, "id": 2073, "job": "Casting", "name": "Gail Stevens"}, {"credit_id": "58c68ef19251411d62016ec3", "department": "Sound", "gender": 2, "id": 3996, "job": "Sound Re-Recording Mixer", "name": "Tom Johnson"}, {"credit_id": "52fe479ac3a36847f813ead7", "department": "Editing", "gender": 2, "id": 6958, "job": "Editor", "name": "Eric Zumbrunnen"}, {"credit_id": "58c69a9ec3a3684

In [4669]:
import ast
msg = ast.literal_eval(msg)

In [4670]:
for i in msg:
    if i['job']=='Director':
        print(i['name'])
        break

Andrew Stanton


In [4671]:
df2['crew'] = df2['crew'].apply(fetch_dir_and_prod)

In [4672]:
df2.head(1)

id  \
0  19995   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [4673]:
##keywords

In [4674]:
msg = df2.iloc[0, 4]

In [4675]:
msg

'[{"id": 1463, "name": "culture clash"}, {"id": 2964, "name": "future"}, {"id": 3386, "name": "space war"}, {"id": 3388, "name": "space colony"}, {"id": 3679, "name": "society"}, {"id": 3801, "name": "space travel"}, {"id": 9685, "name": "futuristic"}, {"id": 9840, "name": "romance"}, {"id": 9882, "name": "space"}, {"id": 9951, "name": "alien"}, {"id": 10148, "name": "tribe"}, {"id": 10158, "name": "alien planet"}, {"id": 10987, "name": "cgi"}, {"id": 11399, "name": "marine"}, {"id": 13065, "name": "soldier"}, {"id": 14643, "name": "battle"}, {"id": 14720, "name": "love affair"}, {"id": 165431, "name": "anti war"}, {"id": 193554, "name": "power relations"}, {"id": 206690, "name": "mind and soul"}, {"id": 209714, "name": "3d"}]'

In [4676]:
def fetch_keyowrds(string):
    ls = []
    for i in  ast.literal_eval(string):
        ls.append(i['name'])
            
    return ls

In [4677]:
fetch_keyowrds(msg)

['culture clash',
 'future',
 'space war',
 'space colony',
 'society',
 'space travel',
 'futuristic',
 'romance',
 'space',
 'alien',
 'tribe',
 'alien planet',
 'cgi',
 'marine',
 'soldier',
 'battle',
 'love affair',
 'anti war',
 'power relations',
 'mind and soul',
 '3d']

In [4678]:
df2['keywords'] = df2['keywords'].apply(fetch_genres)

In [4679]:
df2.head(1)

id  \
0  19995   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [4680]:
##cast (first 3)

In [4681]:
msg = df2.iloc[0, 1]

In [4682]:
msg

'[{"cast_id": 242, "character": "Jake Sully", "credit_id": "5602a8a7c3a3685532001c9a", "gender": 2, "id": 65731, "name": "Sam Worthington", "order": 0}, {"cast_id": 3, "character": "Neytiri", "credit_id": "52fe48009251416c750ac9cb", "gender": 1, "id": 8691, "name": "Zoe Saldana", "order": 1}, {"cast_id": 25, "character": "Dr. Grace Augustine", "credit_id": "52fe48009251416c750aca39", "gender": 1, "id": 10205, "name": "Sigourney Weaver", "order": 2}, {"cast_id": 4, "character": "Col. Quaritch", "credit_id": "52fe48009251416c750ac9cf", "gender": 2, "id": 32747, "name": "Stephen Lang", "order": 3}, {"cast_id": 5, "character": "Trudy Chacon", "credit_id": "52fe48009251416c750ac9d3", "gender": 1, "id": 17647, "name": "Michelle Rodriguez", "order": 4}, {"cast_id": 8, "character": "Selfridge", "credit_id": "52fe48009251416c750ac9e1", "gender": 2, "id": 1771, "name": "Giovanni Ribisi", "order": 5}, {"cast_id": 7, "character": "Norm Spellman", "credit_id": "52fe48009251416c750ac9dd", "gender": 

In [4683]:
def fetch_crew_first_three(string):
    ls = []
    cnt = 3
    for i in  ast.literal_eval(string):
        if cnt==0:
            break
        else:
            ls.append(i['name'])
        cnt=cnt-1
    return ls

In [4684]:
msg

'[{"cast_id": 242, "character": "Jake Sully", "credit_id": "5602a8a7c3a3685532001c9a", "gender": 2, "id": 65731, "name": "Sam Worthington", "order": 0}, {"cast_id": 3, "character": "Neytiri", "credit_id": "52fe48009251416c750ac9cb", "gender": 1, "id": 8691, "name": "Zoe Saldana", "order": 1}, {"cast_id": 25, "character": "Dr. Grace Augustine", "credit_id": "52fe48009251416c750aca39", "gender": 1, "id": 10205, "name": "Sigourney Weaver", "order": 2}, {"cast_id": 4, "character": "Col. Quaritch", "credit_id": "52fe48009251416c750ac9cf", "gender": 2, "id": 32747, "name": "Stephen Lang", "order": 3}, {"cast_id": 5, "character": "Trudy Chacon", "credit_id": "52fe48009251416c750ac9d3", "gender": 1, "id": 17647, "name": "Michelle Rodriguez", "order": 4}, {"cast_id": 8, "character": "Selfridge", "credit_id": "52fe48009251416c750ac9e1", "gender": 2, "id": 1771, "name": "Giovanni Ribisi", "order": 5}, {"cast_id": 7, "character": "Norm Spellman", "credit_id": "52fe48009251416c750ac9dd", "gender": 

In [4685]:
fetch_crew_first_three(msg)

['Sam Worthington', 'Zoe Saldana', 'Sigourney Weaver']

In [4686]:
df2['cast'] = df2['cast'].apply(fetch_crew_first_three)

In [4687]:
df2.head(1)

,id,cast,crew,genres,keywords,original_language,original_title,overview
0,19995,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron],"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, space, alien, tribe, alien planet, cgi, marine, soldier, battle, love affair, anti war, power relations, mind and soul, 3d]",en,Avatar,"In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization."


In [4688]:
df2.rename(columns = {'crew':'director'}, inplace = True)

In [4689]:
df2.head(1)

,id,cast,director,genres,keywords,original_language,original_title,overview
0,19995,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron],"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, space, alien, tribe, alien planet, cgi, marine, soldier, battle, love affair, anti war, power relations, mind and soul, 3d]",en,Avatar,"In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization."


In [4690]:
#Preprocessing 'overview' column

#Lowercasing

df2['overview'] = df2['overview'].str.lower()


# Removing HTML tags

import re

def remove_html(text):
    pattern = re.compile('<.*?>')
    return pattern.sub(r'', text)

df2['overview'] = df2['overview'].apply(remove_html)
# print(df.head())


#Remove @

def remove_at_the_rate(text):

  ls = []
  new = []

  ls = nlp(text)

  for word in ls:
        if word.text != "@":
            new.append(word.text)

  return ' '.join(new)

df2['overview'] = df2['overview'].apply(remove_at_the_rate)



#Removing URL

import re

def remove_url(text):
    pattern = re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub(r'', text)

df2['overview'] = df2['overview'].apply(remove_url)
# print(df.head())


#Remmove punctuation

import string

punc = string.punctuation

def  remove_punc(text):

    return text.translate(str.maketrans('', '', punc))

df2['overview'] = df2['overview'].apply(remove_punc)


#Correcting spelling

# from textblob import TextBlob

# def spell_check(text):

#     textblb = TextBlob(text)
#     return textblb.correct().string

# X['remainder__text'] = X['remainder__text'].apply(spell_check)


from autocorrect import Speller

check = Speller()

def check_spell(text):

  return check(text)

df2['overview'] = df2['overview'].apply(check_spell)


# Removing stop words


from nltk.corpus import stopwords

stopwords = stopwords.words('english')

def remove_stop_words(text):
    ls = []
    new = []

    ls = nlp(text)

    for word in ls:
        if word.text not in stopwords:

            new.append(word.text)

    return ' '.join(new)

df2['overview'] = df2['overview'].apply(remove_stop_words)


#Removing Contradictions

import contractions

def remove_contradictions(text):

  return " ".join([contractions.fix(word.text) for word in nlp(text)])

df2['overview'] = df2['overview'].apply(remove_contradictions)


# #Lemmetization

# from nltk.stem.wordnet import WordNetLemmatizer

# lemmatizer = WordNetLemmatizer()

def Lemmetization(text):

  return " ".join([word.lemma_ for word in nlp(text)])


# #Stemming

# from nltk.stem.porter import PorterStemmer

# ps = PorterStemmer()
# def stemming(text):

#     return " ".join([ps.stem(word.text) for word in nlp(text)])

df2['overview'] = df2['overview'].apply(Lemmetization)




In [4691]:
#Tokenization of overview column

In [4692]:
# from nltk.tokenize import sent_tokenize, word_tokenize

# def token_nlp(string):
#     ls = []
#     # doc1 = nlp(string)
#     doc1 = word_tokenize(string)
#     ls.append(doc1)
#     return ls

In [4693]:
# df2['overview'] = df2['overview'].apply(token_nlp)

In [4694]:
df2.iloc[0, 7]

'22nd century     paraplegic marine dispatch moon pandora unique mission     become torn follow order protect alien civilization'

In [4695]:
df2.head(1)

,id,cast,director,genres,keywords,original_language,original_title,overview
0,19995,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron],"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, space, alien, tribe, alien planet, cgi, marine, soldier, battle, love affair, anti war, power relations, mind and soul, 3d]",en,Avatar,22nd century paraplegic marine dispatch moon pandora unique mission become torn follow order protect alien civilization


In [4696]:
df2['tags'] = df2.sum(axis=1)

In [4697]:
df2.head(1)

,id,cast,director,genres,keywords,original_language,original_title,overview,tags
0,19995,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron],"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, space, alien, tribe, alien planet, cgi, marine, soldier, battle, love affair, anti war, power relations, mind and soul, 3d]",en,Avatar,22nd century paraplegic marine dispatch moon pandora unique mission become torn follow order protect alien civilization,19995


In [4698]:
#Removing spaces between names in cast and crew column

In [4699]:
def remove_spaces(ls):
    for i in range(len(ls)):
        ls[i] = ls[i].replace(' ', '')
    return ls
        # print(ls[i])

In [4700]:
ls1 = df2.iloc[0, 1]

In [4701]:
ls1

['Sam Worthington', 'Zoe Saldana', 'Sigourney Weaver']

In [4702]:
remove_spaces(ls1)

['SamWorthington', 'ZoeSaldana', 'SigourneyWeaver']

In [4703]:
df2.drop(columns='tags', axis=1, inplace=True)

In [4704]:
df2.head(1)

,id,cast,director,genres,keywords,original_language,original_title,overview
0,19995,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[James Cameron],"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, space, alien, tribe, alien planet, cgi, marine, soldier, battle, love affair, anti war, power relations, mind and soul, 3d]",en,Avatar,22nd century paraplegic marine dispatch moon pandora unique mission become torn follow order protect alien civilization


In [4705]:
df2['cast'] = df2['cast'].apply(remove_spaces)

In [4706]:
df2['director'] = df2['director'].apply(remove_spaces)

In [4707]:
# df2['genres'] = df2['genres'].apply(remove_spaces)

In [4708]:
df2.head(1)

,id,cast,director,genres,keywords,original_language,original_title,overview
0,19995,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, space, alien, tribe, alien planet, cgi, marine, soldier, battle, love affair, anti war, power relations, mind and soul, 3d]",en,Avatar,22nd century paraplegic marine dispatch moon pandora unique mission become torn follow order protect alien civilization


In [4709]:
# df2['keywords'] = df2['keywords'].apply(remove_spaces)

In [4710]:
df2['combined'] = df2['cast'] + df2['director'] + df2['genres'] + df2['keywords'] 

In [4711]:
df2.head(1)

,id,cast,director,genres,keywords,original_language,original_title,overview,combined
0,19995,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, space, alien, tribe, alien planet, cgi, marine, soldier, battle, love affair, anti war, power relations, mind and soul, 3d]",en,Avatar,22nd century paraplegic marine dispatch moon pandora unique mission become torn follow order protect alien civilization,"[SamWorthington, ZoeSaldana, SigourneyWeaver, JamesCameron, Action, Adventure, Fantasy, Science Fiction, culture clash, future, space war, space colony, society, space travel, futuristic, romance, space, alien, tribe, alien planet, cgi, marine, soldier, battle, love affair, anti war, power relations, mind and soul, 3d]"


In [4712]:
msg = df2.iloc[0, 7]

In [4713]:
df2.iloc[0, 8]

['SamWorthington',
 'ZoeSaldana',
 'SigourneyWeaver',
 'JamesCameron',
 'Action',
 'Adventure',
 'Fantasy',
 'Science Fiction',
 'culture clash',
 'future',
 'space war',
 'space colony',
 'society',
 'space travel',
 'futuristic',
 'romance',
 'space',
 'alien',
 'tribe',
 'alien planet',
 'cgi',
 'marine',
 'soldier',
 'battle',
 'love affair',
 'anti war',
 'power relations',
 'mind and soul',
 '3d']

In [4714]:
def replace_big_spaces(string):
    string = string.replace('  ', '')
    return string

In [4715]:
replace_big_spaces(msg)

'22nd century paraplegic marine dispatch moon pandora unique mission become torn follow order protect alien civilization'

In [4716]:
df2['overview'] = df2['overview'].apply(replace_big_spaces)

In [4717]:
df2.head(5)

,id,cast,director,genres,keywords,original_language,original_title,overview,combined
0,19995,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, space, alien, tribe, alien planet, cgi, marine, soldier, battle, love affair, anti war, power relations, mind and soul, 3d]",en,Avatar,22nd century paraplegic marine dispatch moon pandora unique mission become torn follow order protect alien civilization,"[SamWorthington, ZoeSaldana, SigourneyWeaver, JamesCameron, Action, Adventure, Fantasy, Science Fiction, culture clash, future, space war, space colony, society, space travel, futuristic, romance, space, alien, tribe, alien planet, cgi, marine, soldier, battle, love affair, anti war, power relations, mind and soul, 3d]"
1,285,"[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski],"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india trading company, love of one's life, traitor, shipwreck, strong woman, ship, alliance, calypso, afterlife, fighter, pirate, swashbuckler, aftercreditsstinger]",en,Pirates of the Caribbean: At World's End,captain barbossa long believe dead come back life head edge earth turner elizabeth swan nothing quite seem,"[JohnnyDepp, OrlandoBloom, KeiraKnightley, GoreVerbinski, Adventure, Fantasy, Action, ocean, drug abuse, exotic island, east india trading company, love of one's life, traitor, shipwreck, strong woman, ship, alliance, calypso, afterlife, fighter, pirate, swashbuckler, aftercreditsstinger]"
2,206647,"[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes],"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi6, british secret service, united kingdom]",en,Spectre,cryptic message bond ’s past send trail uncover sinister organization battle political force keep secret service alive bond feel back layer deceit reveal terrible truth behind spectre,"[DanielCraig, ChristophWaltz, LéaSeydoux, SamMendes, Action, Adventure, Crime, spy, based on novel, secret agent, sequel, mi6, british secret service, united kingdom]"
3,49026,"[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan],"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret identity, burglar, hostage drama, time bomb, gotham city, vigilante, cover-up, superhero, villainess, tragic hero, terrorism, destruction, catwoman, cat burglar, imax, flood, criminal underworld, batman]",en,The Dark Knight Rises,follow death district attorney harvey dent batman assume responsibility dent crime protect late attorney reputation subsequently hunt gotham city police department eight year later batman encounter mysterious selena kyle villainous bane new terrorist leader overwhelm gotham fine dark knight resurface protect city brand enemy,"[ChristianBale, MichaelCaine, GaryOldman, ChristopherNolan, Action, Crime, Drama, Thriller, dc comics, crime fighter, terrorist, secret identity, burglar, hostage drama, time bomb, gotham city, vigilante, cover-up, superhero, villainess, tragic hero, terrorism, destruction, catwoman, cat burglar, imax, flood, criminal underworld, batman]"
4,49529,"[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton],"[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel, princess, alien, steampunk, martian, escape, edgar rice burroughs, alien race, superhuman strength, mars civilization, sword and planet, 19th century, 3d]",en,John Carter,john carter war weary former military captain inexplicably transport mysterious exotic planet bassoon mar reluctantly becomes embroil epic conflict world brink collapse carter rediscovery humanity realize survival bassoon people rest hand,"[TaylorKitsch, LynnCollins, SamanthaMorton, AndrewStanton, Action, Adventure, Science Fiction, based on novel, mars, medallion, space travel, princess, alien, steampunk, martian, escape, edgar rice burroughs

In [4718]:
msg = df2.iloc[0, 7]

In [4719]:
msg

'22nd century paraplegic marine dispatch moon pandora unique mission become torn follow order protect alien civilization'

In [4720]:
def list_to_string(ls):
    st=""
    
    for i in ls:
        st = ' '.join(ls)
    return st

In [4721]:
msg = df2.iloc[0, 8]

In [4722]:
msg

['SamWorthington',
 'ZoeSaldana',
 'SigourneyWeaver',
 'JamesCameron',
 'Action',
 'Adventure',
 'Fantasy',
 'Science Fiction',
 'culture clash',
 'future',
 'space war',
 'space colony',
 'society',
 'space travel',
 'futuristic',
 'romance',
 'space',
 'alien',
 'tribe',
 'alien planet',
 'cgi',
 'marine',
 'soldier',
 'battle',
 'love affair',
 'anti war',
 'power relations',
 'mind and soul',
 '3d']

In [4723]:
list_to_string(msg)

'SamWorthington ZoeSaldana SigourneyWeaver JamesCameron Action Adventure Fantasy Science Fiction culture clash future space war space colony society space travel futuristic romance space alien tribe alien planet cgi marine soldier battle love affair anti war power relations mind and soul 3d'

In [4724]:
df2['combined'] = df2['combined'].apply(list_to_string)

In [4725]:
df2.head(1)

,id,cast,director,genres,keywords,original_language,original_title,overview,combined
0,19995,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, space, alien, tribe, alien planet, cgi, marine, soldier, battle, love affair, anti war, power relations, mind and soul, 3d]",en,Avatar,22nd century paraplegic marine dispatch moon pandora unique mission become torn follow order protect alien civilization,SamWorthington ZoeSaldana SigourneyWeaver JamesCameron Action Adventure Fantasy Science Fiction culture clash future space war space colony society space travel futuristic romance space alien tribe alien planet cgi marine soldier battle love affair anti war power relations mind and soul 3d


In [4726]:
def insert_space_at_beginning(string):
    string = ' ' + string
    return string

In [4727]:
df2['overview']=df2['overview'].apply(insert_space_at_beginning)

In [4728]:
df2['tags'] = df2['combined'] + df2['overview']

In [4729]:
df2.head(1)

,id,cast,director,genres,keywords,original_language,original_title,overview,combined,tags
0,19995,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, space, alien, tribe, alien planet, cgi, marine, soldier, battle, love affair, anti war, power relations, mind and soul, 3d]",en,Avatar,22nd century paraplegic marine dispatch moon pandora unique mission become torn follow order protect alien civilization,SamWorthington ZoeSaldana SigourneyWeaver JamesCameron Action Adventure Fantasy Science Fiction culture clash future space war space colony society space travel futuristic romance space alien tribe alien planet cgi marine soldier battle love affair anti war power relations mind and soul 3d,SamWorthington ZoeSaldana SigourneyWeaver JamesCameron Action Adventure Fantasy Science Fiction culture clash future space war space colony society space travel futuristic romance space alien tribe alien planet cgi marine soldier battle love affair anti war power relations mind and soul 3d 22nd century paraplegic marine dispatch moon pandora unique mission become torn follow order protect alien civilization


In [4730]:
df2.iloc[0, 9]

'SamWorthington ZoeSaldana SigourneyWeaver JamesCameron Action Adventure Fantasy Science Fiction culture clash future space war space colony society space travel futuristic romance space alien tribe alien planet cgi marine soldier battle love affair anti war power relations mind and soul 3d 22nd century paraplegic marine dispatch moon pandora unique mission become torn follow order protect alien civilization'

In [4731]:
df2['original_language'] = df2['original_language'].apply(insert_space_at_beginning)

In [4732]:
df2['tags'] = df2['tags'] + df2['original_language']

In [4733]:
df2.head(1)

,id,cast,director,genres,keywords,original_language,original_title,overview,combined,tags
0,19995,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, space, alien, tribe, alien planet, cgi, marine, soldier, battle, love affair, anti war, power relations, mind and soul, 3d]",en,Avatar,22nd century paraplegic marine dispatch moon pandora unique mission become torn follow order protect alien civilization,SamWorthington ZoeSaldana SigourneyWeaver JamesCameron Action Adventure Fantasy Science Fiction culture clash future space war space colony society space travel futuristic romance space alien tribe alien planet cgi marine soldier battle love affair anti war power relations mind and soul 3d,SamWorthington ZoeSaldana SigourneyWeaver JamesCameron Action Adventure Fantasy Science Fiction culture clash future space war space colony society space travel futuristic romance space alien tribe alien planet cgi marine soldier battle love affair anti war power relations mind and soul 3d 22nd century paraplegic marine dispatch moon pandora unique mission become torn follow order protect alien civilization en


In [4734]:
df2.iloc[0, 9]

'SamWorthington ZoeSaldana SigourneyWeaver JamesCameron Action Adventure Fantasy Science Fiction culture clash future space war space colony society space travel futuristic romance space alien tribe alien planet cgi marine soldier battle love affair anti war power relations mind and soul 3d 22nd century paraplegic marine dispatch moon pandora unique mission become torn follow order protect alien civilization en'

In [4735]:
movies = df2[['id', 'original_title', 'tags']]

In [4736]:
movies.iloc[0,2]

'SamWorthington ZoeSaldana SigourneyWeaver JamesCameron Action Adventure Fantasy Science Fiction culture clash future space war space colony society space travel futuristic romance space alien tribe alien planet cgi marine soldier battle love affair anti war power relations mind and soul 3d 22nd century paraplegic marine dispatch moon pandora unique mission become torn follow order protect alien civilization en'

In [4737]:
# movies['original_title'] = movies['original_title'].apply(remove_spaces_str)

In [4738]:
# movies['original_title'] = movies['original_title'].apply(insert_space_at_beginning)

In [4739]:
# movies['tags'] = movies['tags'] + movies['original_title']

In [4740]:
movies.iloc[2, 2]

'DanielCraig ChristophWaltz LéaSeydoux SamMendes Action Adventure Crime spy based on novel secret agent sequel mi6 british secret service united kingdom cryptic message bond ’s past send trail uncover sinister organization battle political force keep secret service alive bond feel back layer deceit reveal terrible truth behind spectre en'

In [4741]:
msg = movies.iloc[1, 2]

In [4742]:
def remove_spaces_str(string):
    string = string.replace(' ', '')
    return string

In [4743]:
msg

"JohnnyDepp OrlandoBloom KeiraKnightley GoreVerbinski Adventure Fantasy Action ocean drug abuse exotic island east india trading company love of one's life traitor shipwreck strong woman ship alliance calypso afterlife fighter pirate swashbuckler aftercreditsstinger captain barbossa long believe dead come back life head edge earth turner elizabeth swan nothing quite seem en"

In [4744]:
movies.head(1)

,id,original_title,tags
0,19995,Avatar,SamWorthington ZoeSaldana SigourneyWeaver JamesCameron Action Adventure Fantasy Science Fiction culture clash future space war space colony society space travel futuristic romance space alien tribe alien planet cgi marine soldier battle love affair anti war power relations mind and soul 3d 22nd century paraplegic marine dispatch moon pandora unique mission become torn follow order protect alien civilization en


In [4745]:
movies.iloc[1,2]

"JohnnyDepp OrlandoBloom KeiraKnightley GoreVerbinski Adventure Fantasy Action ocean drug abuse exotic island east india trading company love of one's life traitor shipwreck strong woman ship alliance calypso afterlife fighter pirate swashbuckler aftercreditsstinger captain barbossa long believe dead come back life head edge earth turner elizabeth swan nothing quite seem en"

In [4746]:
# movies['original_title'] = movies['original_title'].apply(insert_space_at_beginning)

In [4747]:
#Importing the DNN Model

In [4748]:
!pip install gensim

In [4749]:
import gensim

In [4750]:
from gensim.utils import simple_preprocess

In [4751]:
msg = movies.iloc[1, 2]

In [4752]:
msg

"JohnnyDepp OrlandoBloom KeiraKnightley GoreVerbinski Adventure Fantasy Action ocean drug abuse exotic island east india trading company love of one's life traitor shipwreck strong woman ship alliance calypso afterlife fighter pirate swashbuckler aftercreditsstinger captain barbossa long believe dead come back life head edge earth turner elizabeth swan nothing quite seem en"

In [4753]:
msg = remove_punc(msg)

In [4754]:
from nltk.tokenize import sent_tokenize, word_tokenize

In [4755]:
doc1 = nlp(msg)

In [4756]:
doc1

JohnnyDepp OrlandoBloom KeiraKnightley GoreVerbinski Adventure Fantasy Action ocean drug abuse exotic island east india trading company love of ones life traitor shipwreck strong woman ship alliance calypso afterlife fighter pirate swashbuckler aftercreditsstinger captain barbossa long believe dead come back life head edge earth turner elizabeth swan nothing quite seem en

In [4757]:
#Removing punctuations from movie's titles

In [4758]:
# movies['original_title'] = movies['original_title'].apply(remove_punc)

In [4759]:
# movies['tags'] = movies['tags'] + movies['original_title']

In [4760]:
msg = movies.iloc[1, 2]

In [4761]:
msg

"JohnnyDepp OrlandoBloom KeiraKnightley GoreVerbinski Adventure Fantasy Action ocean drug abuse exotic island east india trading company love of one's life traitor shipwreck strong woman ship alliance calypso afterlife fighter pirate swashbuckler aftercreditsstinger captain barbossa long believe dead come back life head edge earth turner elizabeth swan nothing quite seem en"

In [4762]:
doc1 = sent_tokenize(msg)

In [4763]:
ls = []
for word in doc1:
    ls.append(simple_preprocess(word))


In [4764]:
ls

[['johnnydepp',
  'orlandobloom',
  'keiraknightley',
  'goreverbinski',
  'adventure',
  'fantasy',
  'action',
  'ocean',
  'drug',
  'abuse',
  'exotic',
  'island',
  'east',
  'india',
  'trading',
  'company',
  'love',
  'of',
  'one',
  'life',
  'traitor',
  'shipwreck',
  'strong',
  'woman',
  'ship',
  'alliance',
  'calypso',
  'afterlife',
  'fighter',
  'pirate',
  'swashbuckler',
  'captain',
  'barbossa',
  'long',
  'believe',
  'dead',
  'come',
  'back',
  'life',
  'head',
  'edge',
  'earth',
  'turner',
  'elizabeth',
  'swan',
  'nothing',
  'quite',
  'seem',
  'en']]

In [4765]:

def making_corpus(string):
    corpus = []
    doc1 = sent_tokenize(string)
    for word in doc1:
        corpus.append(simple_preprocess(word))

    return corpus

In [4766]:
msg = movies.iloc[2, 2]

In [4767]:
making_corpus(msg)

[['danielcraig',
  'christophwaltz',
  'léaseydoux',
  'sammendes',
  'action',
  'adventure',
  'crime',
  'spy',
  'based',
  'on',
  'novel',
  'secret',
  'agent',
  'sequel',
  'mi',
  'british',
  'secret',
  'service',
  'united',
  'kingdom',
  'cryptic',
  'message',
  'bond',
  'past',
  'send',
  'trail',
  'uncover',
  'sinister',
  'organization',
  'battle',
  'political',
  'force',
  'keep',
  'secret',
  'service',
  'alive',
  'bond',
  'feel',
  'back',
  'layer',
  'deceit',
  'reveal',
  'terrible',
  'truth',
  'behind',
  'spectre',
  'en']]

In [4768]:
movies['tags'] = movies['tags'].apply(making_corpus)

In [4769]:
movies['tags'].iloc[0]

[['samworthington',
  'zoesaldana',
  'sigourneyweaver',
  'jamescameron',
  'action',
  'adventure',
  'fantasy',
  'science',
  'fiction',
  'culture',
  'clash',
  'future',
  'space',
  'war',
  'space',
  'colony',
  'society',
  'space',
  'travel',
  'futuristic',
  'romance',
  'space',
  'alien',
  'tribe',
  'alien',
  'planet',
  'cgi',
  'marine',
  'soldier',
  'battle',
  'love',
  'affair',
  'anti',
  'war',
  'power',
  'relations',
  'mind',
  'and',
  'soul',
  'nd',
  'century',
  'paraplegic',
  'marine',
  'dispatch',
  'moon',
  'pandora',
  'unique',
  'mission',
  'become',
  'torn',
  'follow',
  'order',
  'protect',
  'alien',
  'civilization',
  'en']]

In [4770]:
corpus = []

In [4771]:
def corpus_formation(ls):
    corpus.append(ls)

In [4772]:
movies['tags'] = movies['tags'].apply(corpus_formation)

In [4773]:
from itertools import chain
flatten_corpus = list(chain.from_iterable(corpus))

In [4782]:
flatten_corpus[0]

['samworthington',
 'zoesaldana',
 'sigourneyweaver',
 'jamescameron',
 'action',
 'adventure',
 'fantasy',
 'science',
 'fiction',
 'culture',
 'clash',
 'future',
 'space',
 'war',
 'space',
 'colony',
 'society',
 'space',
 'travel',
 'futuristic',
 'romance',
 'space',
 'alien',
 'tribe',
 'alien',
 'planet',
 'cgi',
 'marine',
 'soldier',
 'battle',
 'love',
 'affair',
 'anti',
 'war',
 'power',
 'relations',
 'mind',
 'and',
 'soul',
 'nd',
 'century',
 'paraplegic',
 'marine',
 'dispatch',
 'moon',
 'pandora',
 'unique',
 'mission',
 'become',
 'torn',
 'follow',
 'order',
 'protect',
 'alien',
 'civilization',
 'en']

In [4792]:
#Embeddings

In [4776]:
model = gensim.models.Word2Vec(window=10, min_count=1, workers=8, vector_size=300, alpha=0.03, min_alpha=0.0007)

In [4777]:
model.build_vocab(flatten_corpus)

In [4778]:
model.train(flatten_corpus, total_words=model.corpus_total_words, epochs=50)

(11149027, 11663450)

In [4779]:
df2['original_title'].values

array(['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre',
       ..., 'Signed, Sealed, Delivered', 'Shanghai Calling',
       'My Date with Drew'], dtype=object)

In [4780]:
# model.wv.most_similar('Spectre')

In [4783]:
model.wv.get_normed_vectors()

array([[ 0.00115103, -0.04994413,  0.10317981, ...,  0.07386347,
        -0.03163153,  0.00637873],
       [ 0.01749475,  0.00602851, -0.09571884, ..., -0.09759472,
        -0.03645818, -0.05670817],
       [ 0.12230875, -0.02665166, -0.10166907, ...,  0.03695907,
        -0.00787717,  0.03930177],
       ...,
       [-0.05240254,  0.00376262, -0.00513886, ..., -0.04239396,
        -0.02303308,  0.02037739],
       [-0.04758076,  0.00950284, -0.00274844, ..., -0.07661479,
        -0.03824546, -0.02978552],
       [-0.0401764 ,  0.00246283, -0.0373081 , ...,  0.00044206,
        -0.04924559,  0.03860565]], dtype=float32)

In [4785]:
# model.wv.index_to_key

In [4787]:
#Average Word2Vec for each tags

In [4790]:
import math
def average(ls):
    return ls.mean()

In [4791]:
avg = []